#Preparando o ambiente

**Execução 1** - baixando e importanto bibliotecas utilizadas ao decorrer da execução do programa

In [ ]:
# Importando e instalando as bibliotecas que serão utilizadas
feedback_geral = ('*'*100)+'\nImportando e baixando bibliotecas\n'+('*'*100)
from google.colab import drive
from google.colab import output
try:
  try:
    import time
    import os
    import sys
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    import re
    import joblib
  except:
    !pip install requests
    !pip install beautifulsoup4
    !pip install pandas
    !pip install regex
    !pip install joblib
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    import re
    import joblib
  try:
    from urllib import request
    import fitz
  except:
    !pip install urllib3
    from urllib import request
    !pip install PyMuPDF
    import fitz

except: # Caso ocorra algum erro ao instalar os pacotes/bibliotecas seremos notificados com uma mensagem na tela
  output.clear()
  print('\nErro ao instalar pacotes/bibliotecas.')
  execucao = False
  feedback_geral += '\nErro ao instalar pacotes/bibliotecas.'
else:
  execucao = True # Variável que armazenará o sucesso no download das bibliotecas
  output.clear()
  print('='*100)
  print('Bibliotecas carregadas!\nPode executar o código abaixo.')
  print('='*100)
  feedback_geral += '\nBibliotecas carregadas!'

Bibliotecas carregadas!
Pode executar o código abaixo.


**Execução 2** - Conectando ambiente do Google Colab ao Google Drive

In [ ]:
feedback_geral += '\n'+('*'*100)+'\nMontando Drive.\n'+('*'*100)
try:
  drive.mount('/content/drive')
except:
  print('\nErro ao "sincronizar" Google Drive com este ambiente do Google Colab.\nExecute a célula novamente ou tente "montar" o Drive manualmente no botão de "Montar Drive" no menu lateral esquerdo.')
  print('Descrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
  execucao = False
  feedback_geral += '\nErro ao "sincronizar" Google Drive com este ambiente do Google Colab.\nExecute a célula novamente ou tente "montar" o Drive manualmente no botão de "Montar Drive" no menu lateral esquerdo.Descrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
else:
  print('='*100)
  print('Drive conectado com sucesso.\nPode prosseguir na execução das próximas células.')
  print('='*100)
  feedback_geral += '\nDrive conectado com sucesso.'

Mounted at /content/drive
Drive conectado com sucesso.
Pode prosseguir na execução das próximas células.


**Execução 3** - Definindo variáveis e funções que serão utilizadas ao decorrer do programa

In [ ]:
feedback_geral += '\n'+('*'*100)+'\nDefinindo função de criação de diretório e extração de texto de PDF.\n'+('*'*100)

# caminho_pasta_principal = '/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via Web Scraping/Resultados/Teste com a Coleção de História do Repositorio UFSC'
caminho_pasta_principal = '/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via Web Scraping/Otimizações/Resultados/Davi - Otimizações/'
# caminho_pasta_principal = '/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via Web Scraping/Otimizações/Resultados/Coleções Repositorio UFSC'

with open(f'{caminho_pasta_principal}/Feedbacks de execução/Feedback Geral Execução 1.txt','a',encoding='utf-8') as f:
  f.write('')
  f.close()
  print('\nArquivo de feedback geral criado com sucesso, pronto para preenchimento.')
  feedback_geral += '\nArquivo de feedback geral criado com sucesso, pronto para preenchimento.'


try:
  indice_ultima_colecao_iterada = joblib.load(caminho_pasta_principal + '/Feedbacks de execução/ultima_colecao_iterada.joblib')
  indice_ultima_colecao_iterada = int(indice_ultima_colecao_iterada)
except:
  indice_ultima_colecao_iterada = 0
try:
  indice_ultimo_link_iterado = joblib.load(caminho_pasta_principal + '/Feedbacks de execução/ultima_link_iterado.joblib')
  indice_ultimo_link_iterado = int(indice_ultimo_link_iterado)
except:
  indice_ultimo_link_iterado = 0


if execucao:
  headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
                (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}
  timeout = 30

  padrao_data = r'[1-2][0-9]{3}-[0-1][0-9]-[0-3][0-9]' # Padrão data normal: ANO-MÊS-DIA --> 2022-09-07
  op1='|[1-2][0-9]{3}-[0-9]-[0-9]'  # 2022-9-7
  op2='|[1-2][0-9]{3}-[0-9]-[0-3][0-9]' # 2022-9-25
  op3='|[1-2][0-9]{3}-[0-1][0-2]-[0-9]' # 2022-09-7
  op4='|[1-2][0-9]{3}'  # Somente o ano: 1998
  padrao_data += op1 + op2 + op3 + op4
  padrao_ano = r'[0-9]{4}'
  padrao_mes = r'-[0-1][0-9]-|-[0-9]-'
  print('\n')
  print('='*100)
  print('Variáveis para requisições e expressões de datas definidas com sucesso.')
  print('='*100)
  feedback_geral += '\nVariáveis para requisições e expressões de datas definidas com sucesso.'
  try:
    def CriaDiretorio(caminho : str):
      try:
        os.makedirs(caminho, exist_ok=True)
        time.sleep(1)
        if os.path.isdir(caminho):
          print(f'Pasta {caminho} criada com sucesso.')
          return True
        else:
          print(f'Tentando criar pasta {caminho} novamente.')
          time.sleep(3)
          if os.path.isdir(caminho):
            print(f'Pasta {caminho} criada com sucesso.')
            return True
          else:
            print(f'Pasta {caminho} não foi carregada corretamente.')
            execucao = False
            return False
      except:
        print(f'Erro na função de criação e verificação de diretório com caminho {caminho}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.')
        feedback_geral += f'\nErro na função de criação e verificação de diretório com caminho {caminho}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'


    def ExtraiTextoPDF(caminho_arquivo_pdf):
      try:
        doc = fitz.open(caminho_arquivo_pdf)
        texto_principal, notas_de_rodape, ano_capa = process_document(doc)
        # doc.save(f'{doc.name.split(".pdf")[0]}_annotaded.pdf')
        doc.close()
        return True, str(texto_principal), str(notas_de_rodape), ano_capa
      except Exception as e:
        descricao_erro = f"{e.__class__}: {e.args}"
        return False, descricao_erro, '', None

    def process_document(doc):
        texto_principal = ''
        notas_de_rodape = '\nNotas de rodapé:\n'
        ano_capa = extract_ano_capa(doc)
        for numero_pagina in range(len(doc)):
            texto_normal, texto_rodape = process_page(doc[numero_pagina])
            texto_principal += ' '.join(texto_normal).strip() + ' '
            notas_de_rodape += ' '.join(texto_rodape).strip() + ' '
        return extrai_texto(texto_principal), extrai_texto(notas_de_rodape), ano_capa

    def extract_ano_capa(doc):
        for numero_pagina in (0, 1):
            for texto in process_page(doc[numero_pagina])[0]:
                if re.match(r'\b\d{4}\b', texto.strip()):
                    return texto.strip()
        return ''

    def process_page(page):
        blocks = page.get_text("dict", flags=11)["blocks"]
        texto_normal = []
        texto_rodape = []
        for block in blocks:
            for line in block['lines']:
                for span in line['spans']:
                    if is_texto_normal(span, block):
                        texto_normal.append(span['text'])
                        # page.add_underline_annot(fitz.Rect(span['bbox']))
                    elif is_texto_rodape(span, block):
                        texto_rodape.append(span['text'])
                        # page.add_highlight_annot(fitz.Rect(span['bbox']))
        return texto_normal, texto_rodape

    def is_texto_normal(span, block):
        return ((span['size'] >= 10.5 or (span['size'] >= 9.3 and block['bbox'][0] > 150)) and
                (len(span['text'].strip()) >= 3 or len(re.findall(r'[0-9][0-9]|[0-9]', span['text'].strip()[:2])) <= 0) and
                span['text'].strip() != '')

    def is_texto_rodape(span, block):
        return (span['size'] < 10.5 and block['bbox'][0] < 150 and
                (len(span['text'].strip()) >= 3 or len(re.findall(r'[0-9][0-9]|[0-9]', span['text'].strip()[:2])) <= 0) and
                span['text'].strip() != '' and span['size'] > 7.5)

    def extrai_texto(texto_extraido: str):
        return texto_extraido.encode('utf-8',errors='ignore').decode('utf-8')


    def ColetaHTML(link_pagina : str):
      try:
        site = requests.get(link_pagina,headers=headers,timeout=timeout)
        if site.status_code == 200:
          soup = BeautifulSoup(site.content, 'html.parser')
          return True, soup
        else:
          print(f'Problema ao coletar HTML da página {link_pagina}.\nsite.status_code != 200')
          return False, ''
      except:
        print(f'Problema ao coletar HTML da página {link_pagina}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.')
        return False, ''










  except:
    print('Problemas ao definir as funções para esse programa.\nDescrição do erro -->'+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.')
    feedback_geral += '\nProblemas ao definir as funções para esse programa.\nDescrição do erro -->'+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
  else:
    print('\n')
    print('='*100)
    print('Funções criadas com sucesso.\nPode dar continuidade na execução das células abaixo.')
    print('='*100)
else:
  print('\nFalha no carregamento das bibliotecas. Tente executar a primeira célula de códigos novamente.\n')
  feedback_geral += '\nFalha no carregamento das bibliotecas. Tente executar a primeira célula de códigos novamente.'


Arquivo de feedback geral criado com sucesso, pronto para preenchimento.


Variáveis para requisições e expressões de datas definidas com sucesso.


Funções criadas com sucesso.
Pode dar continuidade na execução das células abaixo.


#Execução das ações, de fato

**Execução 4** - Entra link por link em cada coleção e coleta os metadados, entra no link do PDF, baixa o PDF, extrai texto do PDF, exclui arquivo PDF, criar arquivo .txt com o texto extraído e salva arquivo .txt.

In [ ]:
ll = ['1',2,3,4,5]

for contagem,elemento in enumerate(ll[3:]):
  print(contagem,elemento)

0 4
1 5


In [ ]:
feedback_geral += '\n'+('*'*100)+'\nIniciando coleta de links salvos e entrando neles para coletar metadados e textos dos PDFs.\n'+('*'*100)
if execucao:
  start2 = time.time()
  try:
    arquivos_pasta_links = os.listdir(f'{caminho_pasta_principal}/Links dos trabalhos - Repositorio UFSC')
    arquivos_links = []
    for arquivo_links in arquivos_pasta_links:
      if ('AVISOS.txt' not in arquivo_links) and ('ERROS.txt' not in arquivo_links):
        arquivos_links.append(arquivo_links)

    if not arquivos_pasta_links:
      print('\nA pasta "Links dos trabalhos - Repositorio UFSC" está vazia.\n')
      execucao = False
      feedback_geral += '\nA pasta "Links dos trabalhos - Repositorio UFSC" está vazia.'
    else:
      # arquivos_links = arquivos_links[0:5] # LIMITANDO COLEÇÕES !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
      print('\nArquivos de links foram encontrados na pasta "Links dos trabalhos - Repositorio UFSC".')
      feedback_geral += '\nArquivos de links foram encontrados na pasta "Links dos trabalhos - Repositorio UFSC".'
  except:
    print(f'\nErro ao carregar arquivos da pasta "Links dos trabalhos - Repositorio UFSC".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
    execucao = False
    feedback_geral += f'\nErro ao carregar arquivos da pasta "Links dos trabalhos - Repositorio UFSC".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'


  if execucao:
    trabalhos_descartados = {'Data':[],'TCC':[],'Problema PDF':[]}
    numero_total_de_trabalhos = 0
    numero_de_erros_arquivo_txt = 0

    indice_de_partida_colecao = indice_ultima_colecao_iterada

    numeracao = indice_de_partida_colecao
    for arquivo in arquivos_links[indice_de_partida_colecao:]:
      index = arquivo.find('_Links_dos_Trabalhos.csv')
      nome_arquivo = arquivo[:index]

      lista_de_erros_trabalho = {'Erros':[]}
      try:
        df = pd.read_csv(f'{caminho_pasta_principal}/Links dos trabalhos - Repositorio UFSC/{arquivo}',encoding='utf-8', sep=',')
      except:
        print('\nErro ao abrir arquivo dos Links dos trabalhos.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
        lista_de_erros_trabalho['Erros'].append('Erro ao abrir arquivo dos Links dos trabalhos.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
        feedback_geral += '\nErro ao abrir arquivo dos Links dos trabalhos.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
      else:
        lista_links_trabalhos = df["Links trabalhos"]
        # lista_links_trabalhos = lista_links_trabalhos[0:6] # LIMITANDO LINKS/TRABALHOS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

        dic = {'Título':[],'Autor':[],'Resumo':[],'Descrição':[],'Data Repositório':[], 'Data PDF':[], 'Links':[],'Link do PDF':[], 'Texto extraído?':[]}

        indice_de_partida_link = indice_ultimo_link_iterado
        contagem = indice_de_partida_link
        for link in lista_links_trabalhos[indice_de_partida_link:]:
          lista_de_falhas_trabalho = {'Erros':[],'Avisos':[]}

          status_coleta_HTML, soup = ColetaHTML(link)
          if status_coleta_HTML:

    #   Otimizações pararam aqui
    #   Foi adicionado ao dicionário Data Repositório (renomeado) Data PDF (que terá que ser retirado na extração do texto) e Texto extraído? (que o preenchimento deverá seguir o status da extração)

            tabela_info = soup.find('table', class_='ds-includeSet-table')
            tabela_arquivos = soup.find('table', attrs={"class":'ds-table file-list'})

            if (tabela_info == None) or (tabela_arquivos == None):
              print('\nNão foi possível encontrar corretamente as informações gerais do trabalho.')
              lista_de_falhas_trabalho['Avisos'].append(f'Não foi possível encontrar corretamente as informações gerais do trabalho com link: "{link}".\ntabela_info == None or tabela_arquivo == None.')
              feedback_geral += f'\nNão foi possível encontrar corretamente as informações gerais do trabalho com link: "{link}".\ntabela_info == None or tabela_arquivo == None.'
              dic["Data Repositório"].append('NÃO LOCALIZADO')
              dic["Data PDF"].append('NÃO LOCALIZADO')
              dic["Descrição"].append('NÃO LOCALIZADO')
              dic["Link do PDF"].append('NÃO LOCALIZADO')
              dic["Resumo"].append('NÃO LOCALIZADO')
              dic["Texto do PDF"].append('NÃO LOCALIZADO')
              dic["Título"].append('NÃO LOCALIZADO')
              dic["Autor"].append('NÃO LOCALIZADO')
              dic["Links"].append(link)
            else:
              print('\nPegando informações gerais...')
              feedback_geral += '\n\nPegando informações gerais... '
              nao_eh_TCC = True
              data_desejada = False
              titulo = autor = resumo = descricao = url = data = link_pdf = 'NÃO LOCALIZADO'

              for info in tabela_info:
                analise = info.text
                if 'Título' in analise[:(len('Título')+2)]:
                  titulo = analise
                  titulo = titulo[(len('titulo:')+2):].strip()
                elif 'Autor' in analise[:len('Autor')+2]:
                  autor = analise
                  autor = autor[(len('autor:')+2):].strip()
                elif 'Resumo' in analise[:len('Resumo')+2]:
                  resumo = analise
                  resumo = resumo[(len('resumo:')+2):].strip()
                elif 'Descrição' in analise[:len('Descrição')+2]:
                  descricao = analise
                  descricao = descricao[(len('descricao:')+2):].strip()
                  if ('TCC' not in descricao[:len('TCC')+2]) and ('Trabalho de Conclusão de Curso' not in descricao[:len('Trabalho de Conclusão de Curso')+2]):
                    nao_eh_TCC = True
                  else:
                    nao_eh_TCC = False
                elif ('URI' in analise[:len('URI')+2]) or ('URL' in analise[:len('URL')+2]):
                  url = analise
                  url = url[(len('url:')+2):].strip()
                elif 'Data' in analise[:len('Data')+2]:
                    texto_data = analise
                    data = re.search(padrao_data,texto_data)
                    if data:
                      data = data.group(0)
                      mes = re.search(padrao_mes,data)
                      if mes:
                        mes = mes.group(0).replace('-','')
                      ano_data = re.search(padrao_ano,data)
                      if ano_data:
                        ano_data = ano_data.group(0)
                      if ano_data and mes:
                        if (int(ano_data) > 2002) or (int(ano_data) == 2002 and int(mes) >= 7):
                          data_desejada = True
                      elif ano_data:
                        if int(ano_data) > 2002:
                          data_desejada = True

              if data_desejada and nao_eh_TCC:

                  if not (os.path.isdir(f'{caminho_pasta_principal}/{nome_arquivo}/{ano_data}')):
                    try:
                      CriaDiretorio(f'{caminho_pasta_principal}/{nome_arquivo}/{ano_data}')
                    except:
                      print(f'\nErro na hora de chamar a função para criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}".')
                      lista_de_falhas_trabalho['Avisos'].append(f'\nErro na hora de chamar a função para criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}".')
                      feedback_geral += f'\nErro na hora de chamar a função para criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}".'

              if nao_eh_TCC == True:
                pdf_encontrado = False
                print('Pegando informações dos arquivos...')
                feedback_geral += '\nPegando informações dos arquivos...'
                for info_arq in tabela_arquivos:
                  analise = info_arq.text
                  if ('PDF' in analise) and ('.pdf' in analise):
                    print('\t--> Opa, esse aqui tem PDF...')
                    feedback_geral += '\n\t--> Opa, esse aqui tem PDF...'
                    link_pdf = info_arq.find('a')
                    pdf_encontrado = True
                    if 'href="' in str(link_pdf):
                      link_pdf = link_pdf['href']
                      link_pdf = 'https://repositorio.ufsc.br'+link_pdf

                      # Código para extração de texto de PDF (PyMuPDF)
                      try:
                        local_PDF = False
                        if (CriaDiretorio(f'{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}')):
                          local_PDF = True
                        else:
                          print(f'\nErro na hora de chamar a função para criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".')
                          lista_de_falhas_trabalho['Avisos'].append(f'\nErro na hora de chamar a função para criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".')
                          feedback_geral += f'\nErro na hora de chamar a função para criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".'
                      except:
                        print(f'\nErro na criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                        lista_de_falhas_trabalho['Avisos'].append(f'Erro na criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                        feedback_geral += f'\nErro na criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
                      else:
                        if local_PDF and data_desejada:
                          try:
                            caminho_arquivo = f'{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}/arquivoPDF_temporario.pdf'
                            request.urlretrieve(link_pdf, caminho_arquivo)
                          except:
                            print(f'\nProblema ao baixar o PDF do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                            lista_de_falhas_trabalho['Avisos'].append(f'Problema ao baixar o PDF do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                            trabalhos_descartados['Problema PDF'].append(link)
                            feedback_geral += f'\nProblema ao baixar o PDF do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
                            numero_de_erros_arquivo_txt += 1
                          else:
                            print(f'\nPDF temporário do trabalho {contagem+1} baixado com sucesso.')
                            feedback_geral += f'\nPDF temporário do trabalho {contagem+1} baixado com sucesso.'
                            try:
                              # Chamada da função que extrai texto dado um determinado caminho (que foi baixado o pdf que terá o texto extraído)
                              resposta_extracao_texto, conteudo, notas_rodape, ano_capa = ExtraiTextoPDF(caminho_arquivo)

                              if resposta_extracao_texto:
                                try:
                                  with open(f"{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}/PDF_texto.txt", "w") as arq:
                                    arq.write(conteudo)
                                    arq.close()
                                  with open(f"{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}/notas_rodape.txt", "w") as arq:
                                    arq.write(notas_rodape)
                                    arq.close()
                                except:
                                  print(f'Problema ao salvar o arquivo como .txt. do conteúdo do PDF temporário do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                                  lista_de_falhas_trabalho['Avisos'].append(f'Problema ao salvar o arquivo como .txt. do conteúdo do PDF temporário do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                                  trabalhos_descartados['Problema PDF'].append(link)
                                  feedback_geral += f'\nProblema ao salvar o arquivo como .txt. do conteúdo do PDF temporário do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
                                  numero_de_erros_arquivo_txt += 1
                                else:
                                  print('Arquivo .txt com o texto do PDF criado com sucesso!')
                                  feedback_geral += '\nArquivo .txt com o texto do PDF criado com sucesso!'
                              else:
                                print(f'Problema ao salvar o arquivo como .txt. do conteúdo do PDF temporário do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}".\nDescrição do erro --> {conteudo}\n')
                                lista_de_falhas_trabalho['Avisos'].append(f'Problema ao salvar o arquivo como .txt. do conteúdo do PDF temporário do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}".\nDescrição do erro --> {conteudo}.')
                                trabalhos_descartados['Problema PDF'].append(link)
                                feedback_geral += f'\nProblema ao salvar o arquivo como .txt. do conteúdo do PDF temporário do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}".\nDescrição do erro --> {conteudo}.'
                                numero_de_erros_arquivo_txt += 1
                              try:
                                os.remove(caminho_arquivo)
                              except:
                                print(f'Problema ao excluir PDF temporário do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                                lista_de_falhas_trabalho['Avisos'].append(f'Problema ao excluir PDF temporário do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                                feedback_geral += f'\nProblema ao excluir PDF temporário do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
                              else:
                                print('PDF temporário excluído com sucesso.')
                                feedback_geral += '\nPDF temporário excluído com sucesso.'

                            except:
                              print(f'Problema ao abrir/ler o conteúdo do PDF do ano {ano_data} de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}" e/ou salvar o arquivo como .txt.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                              lista_de_falhas_trabalho['Avisos'].append(f'Problema ao abrir/ler o conteúdo do PDF de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}" e/ou salvar o arquivo como .txt.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                              trabalhos_descartados['Problema PDF'].append(link)
                              feedback_geral += f'\nProblema ao abrir/ler o conteúdo do PDF de {nome_arquivo}, trabalho {contagem+1}, com link "{link}" e link do PDF "{link_pdf}" e/ou salvar o arquivo como .txt.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
                              numero_de_erros_arquivo_txt += 1
                            for arquivo_temp in os.listdir(f'{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}'):
                              if arquivo_temp.lower().endswith('.pdf') and not arquivo_temp.lower().endswith('_annotaded.pdf'):
                                caminho_do_arquivo = os.path.join(f'{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}', arquivo_temp)
                                os.remove(caminho_do_arquivo)
                                print(f'Arquivo {arquivo_temp} removido forçadamente, após problema.')
                                feedback_geral += f'\nArquivo {arquivo_temp} removido forçadamente, após problema.'
                        else:
                          print('\nEncontramos uma publicação sem data correta!\n\t--> Descartando publicação do nosso processamento...\n')
                          trabalhos_descartados['Data'].append(link)
                          lista_de_falhas_trabalho["Avisos"].append(f'Foi encontrado uma data incorreta no trabalho {contagem+1} de {nome_arquivo} com link "{link}" do ano {ano_data}. Logo, ele foi descartado do processamento.')
                          feedback_geral += f'\nFoi encontrado uma data incorreta no trabalho {contagem+1} de {nome_arquivo} com link "{link}" do ano {ano_data}. Logo, ele foi descartado do processamento.'
                    # # Estaremos pegando apenas o primeiro PDF !!!
                    # # (alguns trabalhos tem mais de um PDF publicado,
                    # # temos que analisar esse cenário com mais cautela ainda). Esse break é usado para isso..
                    break
                if not pdf_encontrado:
                  print('\nNão foi encontrado um arquivo PDF na área de arquivos da página!\n\t--> Descartando publicação do nosso processamento.')
                  trabalhos_descartados['Problema PDF'].append(link)
                  lista_de_falhas_trabalho['Avisos'].append(f'Não foi encontrado arquivo PDF no trabalho {contagem+1} de {nome_arquivo} com link "{link}" do ano {ano_data}. Logo, ele foi descartado do processamento.')
                  feedback_geral += f'\nNão foi encontrado arquivo PDF no trabalho {contagem+1} de {nome_arquivo} com link "{link}" do ano {ano_data}. Logo, ele foi descartado do processamento.'
                  try:
                    local_PDF = False
                    if (CriaDiretorio(f'{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}')):
                      local_PDF = True
                    else:
                      print(f'\nErro na hora de chamar a função para criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".')
                      lista_de_falhas_trabalho['Avisos'].append(f'\nErro na hora de chamar a função para criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".')
                      feedback_geral += f'\nErro na hora de chamar a função para criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".'
                  except:
                    print(f'\nErro na criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                    lista_de_falhas_trabalho['Avisos'].append(f'Erro na criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                    feedback_geral += f'\nErro na criação da pasta "{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
                  else:
                    print(f'\nPasta do Trabalho {contagem+1} criada com sucesso para armazenar o arquio de AVISOS.')
                    feedback_geral += f'\nPasta do Trabalho {contagem+1} criada com sucesso para armazenar o arquio de AVISOS.'

              else:
                print('\nEncontramos um TCC!\n\t--> Descartando publicação do nosso processamento...\n')
                trabalhos_descartados['TCC'].append(link)
                lista_de_falhas_trabalho["Avisos"].append(f'Foi encontrado um TCC no trabalho {contagem+1} de {nome_arquivo} com link "{link}" do ano {ano_data}. Logo, ele foi descartado do processamento.')
                feedback_geral += f'\nFoi encontrado um TCC no trabalho {contagem+1} de {nome_arquivo} com link "{link}" do ano {ano_data}. Logo, ele foi descartado do processamento.'
              try:
                dic["Data Repositório"].append(data)
              except:
                dic["Data Repositório"].append('NÃO LOCALIZADO')
                lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar DATA do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                feedback_geral += f'\nProblema ao encontrar DATA do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
              try:
                dic["Data PDF"].append(ano_capa)
              except:
                dic["Data PDF"].append('NÃO LOCALIZADO')
              try:
                dic["Descrição"].append(descricao)
              except:
                dic["Descrição"].append('NÃO LOCALIZADO')
                lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar DESCRIÇÃO do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                feedback_geral += f'\nProblema ao encontrar DESCRIÇÃO do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
              try:
                dic["Link do PDF"].append(link_pdf)
              except:
                dic["Link do PDF"].append('NÃO LOCALIZADO')
                lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar LINK DO PDF do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                feedback_geral += f'\nProblema ao encontrar LINK DO PDF do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
              try:
                dic["Resumo"].append(resumo)
              except:
                dic["Resumo"].append('NÃO LOCALIZADO')
                lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar RESUMO do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                feedback_geral += f'\nProblema ao encontrar RESUMO do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
              try:
                dic["Título"].append(titulo)
              except:
                dic["Título"].append('NÃO LOCALIZADO')
                lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar TÍTULO do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                feedback_geral += f'\nProblema ao encontrar TÍTULO do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
              try:
                dic["Autor"].append(autor)
              except:
                dic["Autor"].append('NÃO LOCALIZADO')
                lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar AUTOR do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                feedback_geral += f'\nProblema ao encontrar AUTOR do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
              try:
                dic["Links"].append(link)
              except:
                dic["Links"].append('NÃO LOCALIZADO')
                lista_de_falhas_trabalho["Avisos"].append(f'Problema ao encontrar LINK do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
                feedback_geral += f'\nProblema ao encontrar LINK do trabalho com link "{link}".\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'

              numero_total_de_trabalhos += 1
              # with open(f'{caminho_pasta_principal}/Feedback Geral Execução 2.txt','w',encoding='utf-8') as f:
              #   f.write(feedback_geral)
              #   f.close()
              #   print('\n\nArquivo de feedback geral salvo com sucesso')
              #   feedback_geral = ''

          else:
            print('\nErro de conexão com o site. Possivelmente internet instável/fora do ar ou servidor do repositório fora do ar.')
            lista_de_falhas_trabalho["Erros"].append(f'Erro de conexão com o site do repositório "{link}". Possivelmente internet instável/fora do ar ou servidor do repositório fora do ar.')
            feedback_geral += f'\nErro de conexão com o site do repositório "{link}". Possivelmente internet instável/fora do ar ou servidor do repositório fora do ar.'

          if (len(lista_de_falhas_trabalho['Avisos']) > 0):
            file_path = f'{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}/AVISOS.txt'
            texto = 'AVISOS:\n\n'
            for i in range(len(lista_de_falhas_trabalho['Avisos'])):
              texto = texto + '\n' + lista_de_falhas_trabalho['Avisos'][i]
            with open(file_path, 'w') as f:
              f.write(texto)
              print(f'\nArquivo de avisos salvo com sucesso para coleção {nome_arquivo} e trabalho {contagem+1}.\nVerifique-o!')
              feedback_geral += f'\nArquivo de avisos salvo com sucesso para coleção {nome_arquivo} e trabalho {contagem+1}.\nVerifique-o!'
              f.close()
          if (len(lista_de_falhas_trabalho['Erros']) > 0):
            file_path = f'{caminho_pasta_principal}/{nome_arquivo}/{ano_data}/Trabalho {contagem+1}/ERROS.txt'
            texto = 'ERROS:\n\n'
            for i in range(len(lista_de_falhas_trabalho['Erros'])):
              texto = texto + '\n' + lista_de_falhas_trabalho['Erros'][i]
            with open(file_path, 'w') as f:
              f.write(texto)
              print(f'\nArquivo de erros salvo com sucesso para coleção {nome_arquivo} e trabalho {contagem+1}.\nVerifique-o!')
              feedback_geral += f'\nArquivo de erros salvo com sucesso para coleção {nome_arquivo} e trabalho {contagem+1}.\nVerifique-o!'
              f.close()
          # Alterar caminho adicionando {caminho_pasta_principal}/Feedbacks de execução/Feedback Geral Execução 2.txt
          with open(f'{caminho_pasta_principal}/Feedbacks de execução/Feedback Geral Execução 2.txt','a',encoding='utf-8') as f:
            f.write(feedback_geral)
            f.close()
            print('\n\nArquivo de feedback geral salvo com sucesso')
            feedback_geral = ''
          joblib.dump(contagem,caminho_pasta_principal + '/Feedbacks de execução/ultima_link_iterado.joblib')
          feedback_geral += f'\nÚltimo link da coleção iterado salvo com sucesso: {contagem}.'
          contagem += 1




        try:
          df = pd.DataFrame(dic)
          df.to_csv(f'{caminho_pasta_principal}/{nome_arquivo}/Meta_dados_{numeracao+1}.csv', index=False, encoding='utf-8', sep=',')
        except:
          print(f'\nErro ao fazer download do arquivo .csv para coleção {nome_arquivo}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
          lista_de_falhas_trabalho["Erros"].append(f'Erro ao fazer download do arquivo .csv para coleção {nome_arquivo}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
          feedback_geral += f'\nErro ao fazer download do arquivo .csv para coleção {nome_arquivo}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.'
        else:
          print(f'\nDownload do arquivo .csv da coleção {nome_arquivo} feito com sucesso!')
          feedback_geral += f'\nDownload do arquivo .csv da coleção {nome_arquivo} feito com sucesso!'


      if(len(lista_de_erros_trabalho['Erros'])) > 0:
        file_path = f'{caminho_pasta_principal}/ERRO ao abrir {arquivo}.txt'
        with open(file_path, 'w') as f:
          f.write(lista_de_erros_trabalho['Erros'][0])
          print(f'\nArquivo de erro ao abrir .csv de {arquivo} salvo com sucesso.\nVerifique-o!')
          feedback_geral += f'\nArquivo de erro ao abrir .csv de {arquivo} salvo com sucesso.\nVerifique-o!'
          f.close()

      # Ver como ficaria usando os.path.join(caminho_pasta_principal,f'Ultima_colecao_iterada{nome_arquivo}.txt')
      with open(f'{caminho_pasta_principal}/Feedbacks de execução/Ultima_colecao_iterada_{nome_arquivo}.txt','w',encoding='utf-8') as f:
        f.write(str(numeracao))
        f.close()
        print('\n\nArquivo de última coleção iterada salvo com sucesso!')
        feedback_geral += '\nArquivo de última coleção iterada salvo com sucesso!'

      joblib.dump(numeracao, caminho_pasta_principal + '/Feedbacks de execução/ultima_colecao_iterada.joblib')
      feedback_geral += f'\nÚltima coleção iterada salvo com sucesso: {numeracao}.'
      numeracao += 1
      indice_ultimo_link_iterado = 0

    if (len(trabalhos_descartados['Data']) > 0) or (len(trabalhos_descartados['TCC'])>0) or (len(trabalhos_descartados['Problema PDF'])>0):

      file_path = f'{caminho_pasta_principal}/LISTA DE TRABALHOS NÃO CONTEMPLADOS.txt'

      texto_total = f'\n\nTotal de trabalhos não contemplados: {len(trabalhos_descartados["Data"])+len(trabalhos_descartados["TCC"])+len(trabalhos_descartados["Problema PDF"])}\n\n'
      texto_total += f'Data menor ou igual a 2002: {len(trabalhos_descartados["Data"])}.\n'
      texto_total += f'TCCs: {len(trabalhos_descartados["TCC"])}.\n'
      texto_total += f'Problema PDF(s): {len(trabalhos_descartados["Problema PDF"])}.\n'

      if len(trabalhos_descartados['Data']):
        texto_total += '\nDatas (link da página dos trabalhos):'
        for j in range(len(trabalhos_descartados['Data'])):
          texto_total += '\n' + trabalhos_descartados['Data'][j]
      if len(trabalhos_descartados['TCC']):
        texto_total += '\nTCCs (link da página dos trabalhos):'
        for k in range(len(trabalhos_descartados['TCC'])):
          texto_total += '\n' + trabalhos_descartados['TCC'][k]
      if len(trabalhos_descartados['Problema PDF']):
        texto_total += '\Problema PDF (link da página dos trabalhos):'
        for l in range(len(trabalhos_descartados['Problema PDF'])):
          texto_total += '\n' + trabalhos_descartados['Problema PDF'][l]

      with open(file_path, 'w') as f:
        f.write(texto_total)
        print('\nArquivo de listagem de trabalhos não contemplados salvo com sucesso.\nVerifique-o!')
        feedback_geral += '\nArquivo de listagem de trabalhos não contemplados salvo com sucesso.\nVerifique-o!'
        f.close()

  else:
    print('\n')
    print('!'*100)
    print('\nFalha ao abrir e carregar os arquivos presentes na pasta das coleções.\nTente novamente em alguns minutos.\n')
    print('!'*100)
    feedback_geral += '\n'+('!'*100)+'\nFalha ao abrir e carregar os arquivos presentes na pasta das coleções.\nTente novamente em alguns minutos.\n'+('!'*100)

  end2 = time.time()
  duracao2 = end2 - start2
  print('\n')
  print('='*100)
  print('Número total de trabalhos analisados:',numero_total_de_trabalhos)
  print('Número total de textos coletados:',numero_total_de_trabalhos-(len(trabalhos_descartados['Data'])+len(trabalhos_descartados['TCC'])+len(trabalhos_descartados['Problema PDF'])+numero_de_erros_arquivo_txt))
  print('='*100)
  print('\n')
  print('='*100)
  print('Duração total da segunda execução:',round(duracao2,2))
  print('='*100)
  feedback_geral += '\n'+('='*100)+f'\nNúmero total de trabalhos analisados: {numero_total_de_trabalhos}.\nNúmero total de textos coletados: {numero_total_de_trabalhos-(len(trabalhos_descartados["Data"])+len(trabalhos_descartados["TCC"])+len(trabalhos_descartados["Problema PDF"]))}\nDuração total da segunda execução: {round(duracao2,2)} segundos.\n'+('='*100)

else:
  print('\nFalha no carregamento das bibliotecas e/ou na sincronização e criação de pasta do seu Google Drive. Tente executar a primeira e segunda células de códigos novamente.\n')
  feedback_geral += '\nFalha no carregamento das bibliotecas e/ou na sincronização e criação de pasta do seu Google Drive. Tente executar a primeira e segunda células de códigos novamente.'

with open(f'{caminho_pasta_principal}/Feedbacks de execução/Feedback Geral Execução 2.txt','a',encoding='utf-8') as f:
  f.write(feedback_geral)
  f.close()
  print('\n\nArquivo de feedback geral salvo com sucesso')


Arquivos de links foram encontrados na pasta "Links dos trabalhos - Repositorio UFSC".

Pegando informações gerais...
Pegando informações dos arquivos...
	--> Opa, esse aqui tem PDF...
Pasta /content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via Web Scraping/Otimizações/Resultados/Davi/Coleções Repositorio UFSC/HST/2012/Trabalho 1 criada com sucesso.

PDF temporário do trabalho 1 baixado com sucesso.
Arquivo .txt com o texto do PDF criado com sucesso!
PDF temporário excluído com sucesso.


Arquivo de feedback geral salvo com sucesso

Pegando informações gerais...
Pegando informações dos arquivos...
	--> Opa, esse aqui tem PDF...
Pasta /content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via Web Scraping/Otimizações/Resultados/Davi/Coleções Repositorio UFSC/HST/2013/Trabalho 2 criada com sucesso.

PDF temporário do trabalho 2 baixado com sucesso.
Arquivo .txt com o texto do PDF criado com sucesso!
PDF temporári